<a href="https://colab.research.google.com/github/pareshrchaudhary/numericalmethods/blob/main/DerivativesApplications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applications of Derivatives

## 1) Approximation

### 1.1) Linear Approximation

### 1.2) Quadratic Approximation

### 1.3) Newton's Method

## 2) Graphing and critical points: 1st and 2nd derivative tests

## 3) Optimization: max min problems